In [ ]:
""" Written by Adrian Wilkinson (1-206-240-2656 , adrian_wilkinson@yahoo.com)
        Original version,   March 05, 2016
        Current version,    May, 2023
"""

#from b1_triangles import GenerateTrianglesClass     # Generate enscribed triangle point
#from b2_data_In import array                         # Copy/paste "b1_triangles" into this file
#from b3_data_In import array2                        # Copy/paste "b1_triangles" into this file
#from b2_data_In_4_pnts import array                 # reduced Triangles (small set)
#from b3_data_In_4_pnts import array2                # reduced Triangles (small set)
#from att48_xy_1 import array                         # 48 USA cities
#from att48_xy_2 import array2                        # 48 USA cities
from b1_Qatar import array                           # Qatar
from b2_Qatar import array2                          # Qata
#from b1_waterloo_131 import array                   # XQF131
#from b2_waterloo_131 import array2                   # XQF131##
#from b1_Canada import array
#from b2_Canada import array2
from c_addPlace import AddPlaceClass
from d1_convexHull import ConvexHullClass
from d2_reAssignHullPoints import ReAssignHullPointsClass
from d3_thisLayer import ThisLayerClass
from e_updateArray import UpdateArrayClass
from f_lastOneOrTwoHullPoints import LastOneOrTwoHullPointsClass
from g_FullONION import PopulateOnionClass
from h_closedLoopLayers import ClosedLoopLayersClass
from i_PlotONION_Q import PlotOnionClass
from i_PlotONION_Q_Canada import PlotOnionClass
from j_TSP_LoopCount import CountArrayClass
from k_InnerOuterONION import InnerOuterOnionClass
from m_experiment import PhylisClass
from n_zDecider_C3 import PhylisC_Class
from p_FeedBack import ParamsClass
from q_updateONION import UpdateOnionClass
from r_totDistance import PathTotalClass
from s_plotCompInduct import dx_vs_dci_Class
from t_Segmentation import LR_dx_vs_dci_Class
from u_seg_Dist_CI_AUC import PathSegmentClass
from w_LR import LR_continuousClass
from z_firstPlot import PlotFirstClass
import time


def main():

    plotFirst = PlotFirstClass(array).JustPoints()

    Onion = []
    #print("------ (c) Add Place Identification ------")
    arrPlace = []
    plid = AddPlaceClass(array, arrPlace).PlaceId()
    #print("arrPlace = ", arrPlace)
    cycleChk = len(arrPlace)
    print("Length of arrPlace = ", cycleChk)

    while ( cycleChk > 2 ):

        #print("------ (d1) Create Convex Hull ------")
        layer = []
        nch = ConvexHullClass(array, layer).Around()
        #print("Layer = ", layer)

        #print("------ (d2) ReAssign Hull Points  &&  build Onion ------")
        pnts_shift = ReAssignHullPointsClass(arrPlace, layer, Onion).consistentHullPoints()
        #print("Sorted Layer = ", Onion)

        #print("------ (d3) Sorted Convex Hull ------")
        nxt_layer = ThisLayerClass(array, layer).descSortLayer()
        #print("Sorted Layer = ", layer)

        #print("------ (e) Update Array ------")
        nxt_array = UpdateArrayClass(array, arrPlace, layer).reduceArrays()
        #print("Updated array = ", array)
        #print("Updated PLACE array = ", arrPlace)

        cycleChk = len(arrPlace)
        #print("Length of arrPlace = ", cycleChk)


    if ( cycleChk > 0):
        #print("------ (f) The Last One or Two Points ------")
        last_1_or_2_pnts = LastOneOrTwoHullPointsClass(arrPlace, Onion).last_1_2_HullPoints()
        cycleChk = len(arrPlace)


    #print("Constructed ONION = ", Onion)
    onionLen = len(Onion)
    print("Length of Onion now at this time = ", onionLen)


    #print("------ (h) Populate ONION Layers ------")
    #print("------ (c) Add Place Identification ------")
    transfer = []
    arrPlace = []
    plid = AddPlaceClass(array2, arrPlace).PlaceId()
    #print("arrPlace = ", arrPlace)
    cycleChk = len(arrPlace)
    #print("Length of arrPlace = ", cycleChk)
    #print("Constructed ONION222 = ", Onion)
    #print("Length of Onion now at this time = ", onionLen)
    popOnion = PopulateOnionClass(Onion, arrPlace, transfer).DeLayerOnion()
    ONION = popOnion
    lengthONION = len(ONION)
    #print("The length of ONION is ", lengthONION)
    #print("Full ONION /////////////////= ", ONION)
    NowLetsCount = CountArrayClass(ONION).Question() # Number of merge decisions req'd

    #print("------ (i) Close the Layer Loops of This ONION ------")
    #closeLoops = ClosedLoopLayersClass(ONION).LayerLoopsClosed()

    #print("------ (j) Plot This ONION ------")
    plotOnion = PlotOnionClass(ONION).PlotThisOnion()    # Toggle to see layered onion

    # ******************************** MaxLimit ************************************
    Limiter = 0.0       #  Do NOT change
    maxLimit = 1178
    #  Change based on dataset title
                        # also change in "s_plotCompInduct"  Line 56
                        #  57 for 6 nested triangles,     96 for 48 USA cities
                        #  1178 for Qatar,   746 for XQF131,    368,497 for Canada
    # ******************************** MaxLimit ************************************
    
    noMerges = 0
    calcsCount = 0
    TotalCount = 0.0
    rA=rB=rC1=rC2=rC3=rC4=rD1=rD2=rD3=rD4=rD5=rD_angle =  1.0   # initial settings
    FB_cnt = 0
    rD_angl = 1.0
    
    paramsIN = [rA,
                rB,
                rC1, rC2, rC3, rC4,
                rD1, rD2, rD3, rD4, rD5,
                rD_angl,
                FB_cnt]
   
    FeedBACK = ParamsClass(paramsIN).updateParams()   # initial call
    accumDelta_L = []
    accumComp_Induct = []
    

    while (Limiter < maxLimit):
        #print("------ (k) Inner & Outer of This ONION ------")
        in_out_Onion = InnerOuterOnionClass(ONION).InnerOuterOnion()
        InOut = in_out_Onion
        inner = InOut[0]
        outer = InOut[1]
        #print(inner)
        #print(outer)

        #print("------ (m) Generate Merge Data:  distances, angles ------")
        #calcsMerge = 0
        
        # FEEDBACK LOOP
        FeedBACK = ParamsClass(paramsIN).updateParams()
        #print("Returned and updated params = ", paramsBACK)
        
        #genData = PhylisClass(inner, outer, calcsCount).experiment()
        genData = PhylisClass(inner, outer, calcsCount).experiment_xx()
        #print("888888888888888888888888888 = ", genData)
  
        #print("Toggle No. 3, Distance citeria")
        #print("-- (m_zDecider_C) Merge Decision C: Minimized Complexity Induction Approach --")
        decide = PhylisC_Class(genData, FeedBACK).shortDist()
        #print("DECIDE CCCCCCCCCCC = ", decide)
           
        delta_L = decide[3]
        accumDelta_L.append(delta_L)
        #print("AccumDelta_L = ", accumDelta_L)
        compInduct = decide[4]        
        accumComp_Induct.append(compInduct)
        #print("AccumComp_Induct = ", accumComp_Induct)
  
        #print("------ (n) Update This ONION ------")
        Quacky = ONION
        #upDate = UpdateOnionClass(Quacky, minMerge, noMerges).updateOnion()
        upDate = UpdateOnionClass(Quacky, decide, noMerges).updateOnion()
        ONION = upDate[0]
        noMerges = upDate[1]

        #print("------ (j) Plot This ONION ------")
        # UNCOMMENT THIS to get Interim (Progress) Plots  (difficult on Jupyter?)
        #plotOnion = PlotOnionClass(ONION).PlotThisOnion()

        #***************************************************************
        #print("------ (i) Close the Layer Loops of This ONION ------")
        #closeLoops = ClosedLoopLayersClass(ONION).LayerLoopsClosed()

        Limiter += 1

        
    #print("AccumDelta_L  decide[3] = ", accumDelta_L)   # THISSSSSSSSSSSSSSSSSSSSSSSSSS   
    #print("AccumComp_Induct decide [4] = ", accumComp_Induct) # THISSSSSSSSSSSSSSSSSSSSSSSSSS
    
    print("------ (j) Plot This ONION ------")
    plotOnion = PlotOnionClass(ONION).PlotThisOnion()
    #print("TSP Onion = ", ONION)
    print("Limiter (total number of merges permitted) = ", maxLimit)
    print("Total number of merges required = ", noMerges)
    print("Total number of calculations = ", TotalCount)

    thisPath = PathTotalClass(ONION).PathCalc()
    print("Path total distance = ", thisPath[0], "units")
    print("TRAIL (processed, start set to point 1) = ", thisPath[1])
    #print("Point vs Accrued Distance = ", thisPath[2])
    accumDist = thisPath[2][1]
    
    #print("Accumulative Distance = ", accumDist)
    
    plotComplex = dx_vs_dci_Class(accumDelta_L, accumComp_Induct).compInduct()

    Complex_q = plotComplex[0]       # Delta_L accum
    Complex_r = plotComplex[1]       # Comp_induc accum
    Complex_auc = plotComplex[2]     # Area Under Curve Accum
    #************  Segmenting  [HAND INPUT BREAK POINTS] BELOW **************
    #LR_seg = LR_dx_vs_dci_Class(Complex_q, Complex_r).Seg_b4_LR()
    #************  Segmenting  [HAND INPUT BREAK POINTS] BELOW **************
    #LR_seg_brackets = LR_seg[0]
    #LR_seg_endPts = LR_seg[1]
    #*********** Comment Out the following except for "Qatar" ***************
    #LR_seg_d_ci_auc = PathSegmentClass(LR_seg_endPts, Complex_q, Complex_r, Complex_auc).SegPathCalc()
    # Calculate Delta_L accum, Comp_Induct accum & AUC accum  ***************
    # for each Linear Regression slope region                 ***************
    #LR_plot = LR_continuousClass(LR_seg, LR_seg_d_ci_auc).LR_plot() # Qatar,Find progress data
    #************* Comment Out the above except for "Qatar" *****************
    
if __name__ == "__main__":
        main()